<a href="https://colab.research.google.com/github/hipanky/EDA/blob/master/chatbot_with_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U langgraph langchain_openai langchain langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.4/501.4 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 1.2.13
    Uninstalling langchain-core-1.2.13:
      Successfully uninstalled langchain-core-1.2.13
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.7
    Uninstalling langgraph-prebuilt-1.0.7:
      Successfully uninstalled langgraph-prebuilt-1.0.7
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.8
    Uninstalling langgraph-1.0.8:
      Successfully uninstalled langgraph-1.0.8


In [ ]:
from langchain_core.messages import HumanMessage,BaseMessage,AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from typing import List,Annotated,Union
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

In [ ]:
gpt_4o = ChatOpenAI(model='gpt-4o-mini',
         api_key='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',
         base_url='https://openrouter.ai/api/v1')



In [ ]:
class Chatbot(TypedDict):
    messages : Annotated[List[BaseMessage],add_messages]


In [ ]:
def chat(State:Chatbot):
  messages = State['messages']
  # print(messages)
  response = gpt_4o.invoke(messages).content
  # print(f'response is {response}')

  return {'messages':[response]}


In [ ]:
checkpointer = MemorySaver()

g1 = StateGraph(Chatbot)
g1.add_node(chat)
g1.add_edge(START,'chat')
g1.add_edge('chat',END)
wk1 = g1.compile(checkpointer=checkpointer)



In [ ]:
thread_id = '1'

while(True):
  query = input('Enter your question  ')
  print(f'Your query is -> { query}')

  if query.strip().lower() in ['bye','exit','quit']:
    break

  config = {'configurable':{'thread_id':thread_id}}


  response = wk1.invoke({'messages': [HumanMessage(content=query)]},config=config)
  # print(response)

  print(f'Your Answer is {response['messages'][-1].content} ')


Enter your question  hi my name is pankaj
Your query is -> hi my name is pankaj
Your Answer is Hi again, Pankaj! How can I help you today? 
Enter your question  what is 2+2
Your query is -> what is 2+2
Your Answer is 2 + 2 equals 4. If you have any more questions, feel free to ask! 
Enter your question  add it to 1
Your query is -> add it to 1
Your Answer is If you add 1 to 2 + 2 (which is 4), the result is 5. So, 2 + 2 + 1 equals 5. 
Enter your question  what is the fastest database 
Your query is -> what is the fastest database 
Your Answer is The performance of a database can depend on various factors, including the specific use case, the size of the data, the type of queries, and the hardware it runs on. However, here are some databases that are often cited for their speed under certain conditions:

1. **Redis**: An in-memory data structure store that is often used for caching and real-time analytics. Its speed comes from storing data in-memory rather than on disk.

2. **Aerospike*